# CEIA - NLP Desafío N°2
## Villanueva Cecilia Azul




### Librerías

In [39]:
import pandas as pd
import numpy as np    

import nltk
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

import multiprocessing
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE    

import plotly.graph_objects as go
import plotly.express as px


### Enunciado

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

### Datos y Preprocesamiento

In [40]:
nltk.download('gutenberg')
nltk.download('punkt')  # Necesario para el tokenizador de oraciones
nltk.download('punkt_tab')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Consulto todos los textos que están disponibles en el corpus de Gutenberg.

In [41]:
available_texts = gutenberg.fileids()
print(available_texts)

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


Selecciono **Alice's Adventures in Wonderland de Lewis Carroll**,es una novela fantástica que narra la historia de Alicia, una niña que sigue a un conejo blanco con un reloj en su bolsillo y acaba cayendo por un agujero en el suelo, lo que la lleva a un mundo mágico y surrealista lleno de personajes extraños

In [42]:
# Obtener las oraciones del texto de Moby Dick
sentences = gutenberg.sents("carroll-alice.txt")

# Tokenizar en oraciones
sentences_tk = [[word.lower() for word in sent if word.isalpha()] for sent in sentences]

sentences_tk[:3]

[['alice', 's', 'adventures', 'in', 'wonderland', 'by', 'lewis', 'carroll'],
 ['chapter', 'i'],
 ['down', 'the', 'rabbit', 'hole']]

In [43]:
print("Cantidad de documentos:", len(sentences_tk))

Cantidad de documentos: 1703


###  Word2vec

In [44]:

# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información

class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [45]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,# dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=4,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [46]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentences_tk)

In [47]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1703


In [48]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 687


### Entrenar embeddings

In [49]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentences_tk,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()],
                 )

Loss after epoch 0: 84207.53125
Loss after epoch 1: 58511.765625
Loss after epoch 2: 59441.203125
Loss after epoch 3: 59071.671875
Loss after epoch 4: 58065.890625
Loss after epoch 5: 59363.59375
Loss after epoch 6: 40704.53125
Loss after epoch 7: 58337.5
Loss after epoch 8: 55754.625
Loss after epoch 9: 39616.625
Loss after epoch 10: 54386.0
Loss after epoch 11: 53701.9375
Loss after epoch 12: 55099.9375
Loss after epoch 13: 53326.875
Loss after epoch 14: 52009.125
Loss after epoch 15: 51625.9375
Loss after epoch 16: 50593.25
Loss after epoch 17: 34161.5625
Loss after epoch 18: 35369.625
Loss after epoch 19: 48760.3125
Loss after epoch 20: 45292.375
Loss after epoch 21: 31264.125
Loss after epoch 22: 45839.125
Loss after epoch 23: 45121.375
Loss after epoch 24: 43661.75
Loss after epoch 25: 43559.125
Loss after epoch 26: 43159.125
Loss after epoch 27: 43413.0
Loss after epoch 28: 43546.125
Loss after epoch 29: 42997.5
Loss after epoch 30: 41760.5
Loss after epoch 31: 42842.375
Loss af

(1553930, 2733300)

### Analisis de similaridad de palabras

Selecciono palabras de interés (según las más frecuentes en el libro) para analizar con cuáles se relacionan más (o menos)

In [ ]:
stop_words = set(stopwords.words('english'))

# Palabras mas frecuentes
top_words = w2v_model.wv.index_to_key

# Filtro stopwords de las palabras mas frecuentes
filtered_top_words = [word for word in top_words if word not in stop_words]

print(filtered_top_words[:25])	


['said', 'alice', 'little', 'one', 'know', 'like', 'would', 'went', 'could', 'queen', 'thought', 'time', 'see', 'well', 'king', 'turtle', 'began', 'mock', 'way', 'hatter', 'quite', 'gryphon', 'think', 'rabbit', 'say']


Palabras más similares:

In [51]:
#se configura palabra claves  en el espacio de embeddings
words_of_interest = ["alice", "queen", "time", "rabbit"]

for word in words_of_interest:
    print(f"Most similar to '{word}':")
    print(w2v_model.wv.most_similar(word, topn=5))
    print("\n")


Most similar to 'alice':
[('she', 0.438062846660614), ('slowly', 0.34670165181159973), ('politely', 0.3410396873950958), ('dare', 0.33107230067253113), ('feeling', 0.3297099769115448)]


Most similar to 'queen':
[('reply', 0.4364963471889496), ('ear', 0.403495192527771), ('top', 0.40073999762535095), ('hearts', 0.379926860332489), ('exclaimed', 0.36524277925491333)]


Most similar to 'time':
[('minutes', 0.456287145614624), ('silence', 0.41208967566490173), ('rest', 0.4016607701778412), ('clock', 0.38771218061447144), ('speaking', 0.3818735182285309)]


Most similar to 'rabbit':
[('kid', 0.5864681005477905), ('white', 0.543060839176178), ('chorus', 0.44006937742233276), ('himself', 0.43324965238571167), ('trembling', 0.4202815890312195)]




Palabras menos similares:

In [ ]:
for word in words_of_interest:
    print(f"Least similar to '{word}':")
    print(w2v_model.wv.most_similar(negative=word, topn=5))
    print("\n")

Least similar to 'alice':
[('wow', 0.021753162145614624), ('will', -0.012397035956382751), ('among', -0.01427757740020752), ('heads', -0.028268881142139435), ('cut', -0.03530262038111687)]


Least similar to 'queen':
[('wow', 0.07792161405086517), ('larger', 0.01226002536714077), ('timidly', -0.009457527659833431), ('remember', -0.010450463742017746), ('sort', -0.015757063403725624)]


Least similar to 'time':
[('makes', 0.03729119896888733), ('wow', 0.013288663700222969), ('watch', 0.012097852304577827), ('replied', 0.009812556207180023), ('key', -0.004337225575000048)]


Least similar to 'rabbit':
[('day', 0.03199528902769089), ('would', -0.0010914934100583196), ('life', -0.01835947483778), ('my', -0.018631357699632645), ('puzzled', -0.019626185297966003)]




Si bien se observan palabras cuya relacion tiene sentido en el libro : rabbit - white  / time - minutes  / time - clock / queen - hearts también se puede ver que hay palabras que no están relacionadas con las seleccionadas. Esto tiene sentido porque como contexto se seleccionaron sólo 2 palabras antes y despues de las palabras leidas por el modelo.

### Visualizar agrupación de vectores

In [53]:
def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [56]:
vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=100

fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() 

In [55]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() 

Podría ser beneficioso tunear distintos hiperparametros como la ventana de contexto(window), frecuencia mínima de palabra para incluirla en el vocabulario(min_count), el numero de negativos(negative), y tambien la dimensionalidad de los vectores(vector_size).